In [ ]:
!pip install nltk
!pip install scikit-learn
!pip install PyPDF2
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
  

In [ ]:
import os
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
import PyPDF2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from googletrans import Translator

    # num_sentences para seleccionar solo las oraciones más relevantes y summary_length para limitar el resumen al # de palabras
def generate_summary(text, num_sentences=90, summary_lenght=350):
    # Tokenización de oraciones
    sentences = sent_tokenize(text)

    # Tokenización de palabras y eliminación de palabras vacías (stopwords)
    stop_words = set(stopwords.words("spanish"))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Cálculo de la matriz TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Cálculo de las similitudes entre oraciones
    sentence_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Creación del grafo de similitudes de oraciones
    sentence_graph = nx.from_numpy_array(sentence_similarities)

    # Aplicación del algoritmo de TextRank para obtener las oraciones más relevantes
    scores = nx.pagerank(sentence_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Selección de las primeras num_sentences oraciones más relevantes
    top_sentences = [s[1] for s in ranked_sentences[:num_sentences]]

    # Generación del resumen con las oraciones más relevantes
    summary = ' '.join(top_sentences)

    return summary

# Extraer texto del PDF
pdf_path = os.environ.get("pdf_path", input("Ingrese la ruta del archivo"))  # Ruta al archivo PDF
output_file1 = input("Ingrese donde desea guardar el archivo")
output_file = output_file1 + "/resumen.txt"
articulo_texto = extract_text(pdf_path)
articulo_texto = articulo_texto.replace("[ edit ]", " ")

""" ---> opcion para traducir
translator = Translator()
translation = translator.translate(articulo_texto, src="es", dest="ja") # -----------> src=idioma original - dest= idioma al que queremos traducir
translated_text = translation.text"""

# Generación del resumen
palabras_clave = generate_summary(articulo_texto)

# Guardar el resumen en un archivo de texto
with open(output_file, "w") as file:
    file.write(palabras_clave)

print("Resumen del texto")
print("------------------")
print("El resumen se ha guardado en el archivo:", output_file)

# Ruta al archivo de texto
txt_file = "./resumen.txt"

# Leer el archivo de texto y quitar los espacios en cada línea
with open(txt_file, "r") as file:
    lines = [line.replace(" ", " ") for line in file.readlines()]

# Eliminar los puntos en cada línea
lines = [line.replace(".", "") for line in lines]

# Eliminar espacios adicionales cuando haya más de 4 espacios consecutivos
lines = [re.sub(r"\s{5,}", "", line) for line in lines]

# Sobrescribir el archivo con el texto sin espacios
with open(txt_file, "w") as file:
    file.write("\n".join(lines))

# Leer el archivo de texto y unir todas las líneas en párrafos
with open(txt_file, "r") as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]

# Unir las líneas en párrafos
paragraphs = " ".join(" ".join(lines[i:i+2]) for i in range(0, len(lines), 2))

# Sobrescribir el archivo con los párrafos
with open(txt_file, "w") as file:
    file.write(paragraphs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Ingrese la ruta del archivo/content/CV.pdf
Ingrese donde desea guardar el archivo.
Resumen del texto
------------------
El resumen se ha guardado en el archivo: ./resumen.txt
